
### FXOptions Class Payoffs 

#### 1. Foreign Equity Call Struck in Foreign Currency
**Payoff Formula:** 
$$ \text{Payoff}_{\text{Foreign Call (Foreign Currency)}} = \max\left(S_T - K, 0\right) \cdot X_T $$

**Where:**
- **$S_T$:** Price of the foreign equity at maturity \( $T$ \).
- **$K$:** Strike price of the option.
- **$X_T$:** Exchange rate at maturity \( T \) (the value of the domestic currency in terms of the foreign currency).

#### 2. Foreign Equity Call Struck in Domestic Currency or Compo
**Payoff Formula:**
$$ \text{Payoff}_{\text{Foreign Call (Domestic Currency)}} = \max\left(X_T \cdot S_T - K, 0\right) $$

**Where:**
- **$S_T$:** Price of the foreign equity at maturity \( $T$ \).
- **$X_T$:** Exchange rate at maturity \( $T$ \) (the value of the domestic currency in terms of the foreign currency).
- **$K$:** Strike price of the option in domestic currency.

#### 3. Fixed Foreign Equity Rate Call or Quanto
**Payoff Formula:**
$$ \text{Payoff}_{\text{Fixed Foreign Call}} = \max\left(S_T - K, 0\right) \cdot X_{\text{quanto}} $$

**Where:**
- **$S_T$:** Price of the foreign equity at maturity \( $T$ \).
- **$K$:** Strike price of the option.
- **$X_{\text{quanto}}$:** Quanto adjustment factor or fixed exchange rate used to convert the payoff into the domestic currency.

#### 4. Equity Linked Foreign Exchange Call or ELF X or FX Option Denoted in Domestic Currency
**Payoff Formula:**
$$ \text{Payoff}_{\text{Equity Linked FX Call}} = \max\left(X_T - K, 0\right) \cdot S_T $$

**Where:**
- **$X_T$:** Exchange rate at maturity \( $T$ \) (the value of the domestic currency in terms of the foreign currency).
- **$K$:** Strike price of the option in domestic currency.
- **$S_T$:** Price of the foreign equity at maturity \( $T$ \).

In [1]:
import numpy as np
from scipy.stats import norm
from math import pi

In [2]:
class FXOptions:
    
    def __init__(self,S_t,X_t,K,rd,rf,sigma_s,D_s,T,sigma_x,rho,X_quanto):
        self.S_t = S_t
        self.X_t = X_t
        self.K = K
        self.rd = rd
        self.rf = rf
        self.sigma_s = sigma_s
        self.sigma_x = sigma_x
        self.D_s = D_s
        self.T = T
        self.X_quanto = X_quanto
        self.rho = rho
        
        
    def Foreign_Equity_Call_Struck_In_Foreign_Currency(self):
        
        d_1 = (np.log(self.S_t / self.K) + ((self.rf - self.D_s) + (np.power(self.sigma_s,2)/ 2) * self.T )) / (self.sigma_s * np.sqrt(self.T))
        d_2 = (np.log(self.S_t / self.K) + ((self.rf - self.D_s) - (np.power(self.sigma_s,2)/ 2) * self.T )) / (self.sigma_s * np.sqrt(self.T))
        
        return self.X_t * (self.S_t * np.exp(-self.D_s * self.T) * norm.cdf(d_1) - self.K * np.exp(-self.rf * self.T) * norm.cdf(d_2))
    
    
    
    def Foreign_Equity_Call_Struck_In_Domestic_Currency_or_Compo(self):
        
        
        sigma_g = np.sqrt(np.power(self.sigma_x,2) + 2 * self.rho * self.sigma_x * self.sigma_s + np.power(self.sigma_s,2))
        
        d_1 = (np.log((self.X_t * self.S_t) / self.K) + ((self.rd - self.D_s) + (np.power(sigma_g,2)/ 2) * self.T )) / (sigma_g * np.sqrt(self.T))
        
        d_2 = (np.log((self.X_t * self.S_t) / self.K) + ((self.rd - self.D_s) - (np.power(sigma_g,2)/ 2) * self.T )) / (sigma_g * np.sqrt(self.T))
        
        
        return (self.X_t * self.S_t) * np.exp(-self.D_s * self.T) * norm.cdf(d_1) - self.K * np.exp(-self.rd * self.T) * norm.cdf(d_2)
    
    
    def Fixed_Foreign_Equity_Rate_call_or_Quanto(self):
        
        D_h = self.rd - self.rf + self.D_s * self.rho * self.sigma_x * self.sigma_s
        
        d_1 = (np.log(self.S_t / self.K) + ((self.rd - D_h) + (np.power(self.sigma_s,2)/ 2) * self.T )) / (self.sigma_s * np.sqrt(self.T))
        
        d_2 = (np.log(self.S_t / self.K) + ((self.rd - D_h) - (np.power(self.sigma_s,2)/ 2) * self.T )) / (self.sigma_s * np.sqrt(self.T))
        
        return self.X_quanto * (self.S_t * np.exp(-D_h * self.T) * norm.cdf(d_1) - self.K * np.exp(-self.rd * self.T) * norm.cdf(d_2))
    
    
    def Equity_Linked_Foreign_Exchange_Call_or_Elf_X_or_Fx_Option_Denoted_in_Domestic_Currency(self):
        
        r_u = self.rd - self.rf + self.D_s * self.rho * self.sigma_x * self.sigma_s
        
        d_1 = (np.log(self.X_t / self.K) + ((r_u - self.D_s) + (np.power(self.sigma_x,2)/ 2) * self.T )) / (self.sigma_x * np.sqrt(self.T))
        
        d_2 = (np.log(self.X_t / self.K) + ((r_u - self.D_s) - (np.power(self.sigma_x,2)/ 2) * self.T )) / (self.sigma_x * np.sqrt(self.T))
        
        
        return self.S_t * (self.X_t * np.exp(-self.D_s * self.T * norm.cdf(d_1)) - self.K * np.exp(-r_u * self.T) * norm.cdf(d_2))
    
    
    def shoot_display(self):
        
        import pandas as pd
        
        return pd.DataFrame(data={'Valeur_Foreign_Equity_Call_Struck_In_Foreign_Currency':[self.Equity_Linked_Foreign_Exchange_Call_or_Elf_X_or_Fx_Option_Denoted_in_Domestic_Currency()],
                                 'Valeur_Fixed_Foreign_Equity_Rate_call_or_Quanto':[self.Fixed_Foreign_Equity_Rate_call_or_Quanto()],
                                 'Valeur_Foreign_Equity_Call_Struck_In_Domestic_Currency_or_Compo':[self.Foreign_Equity_Call_Struck_In_Domestic_Currency_or_Compo()],
                                 'Valeur_Foreign_Equity_Call_Struck_In_Foreign_Currency':[self.Foreign_Equity_Call_Struck_In_Foreign_Currency()]}).T.rename(columns={0:'Valeurs'})

In [3]:
option = FXOptions(100,0.20,75,0.3,0.1,0.4,0.05,1,0.25,0.80,0.45)

In [4]:
option.shoot_display()

,Valeurs
Valeur_Foreign_Equity_Call_Struck_In_Foreign_Currency,6.157756
Valeur_Fixed_Foreign_Equity_Rate_call_or_Quanto,12.769980
Valeur_Foreign_Equity_Call_Struck_In_Domestic_Currency_or_Compo,0.327206
